# WCS issues

## PV polynomials

In [1]:
import numpy as np

from copy import copy

from astropy.io import fits
from astropy.wcs import WCS
import sip_tpv as stp
from esutil import wcsutil

from cs_util import canfar

In [2]:
# Get image

id = "2105209"
image_name = f"{id}p.fits.fz"
vos_dir = f"vos:cfis/pitcairn/"
canfar.download(f"{vos_dir}/{image_name}", image_name, verbose=True)

Target file 2105209p.fits.fz exists, skipping download.


In [3]:
# Open file
dat = fits.open(image_name)

In [4]:
# Get main image header
header_0 = dat[0].header

In [5]:
# Get WCS, warning might be raised
wcs_0 = WCS(header_0)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [6]:
# Show PV polynomials: empty with standard "TAN" convention
wcs_0.wcs.get_pv()

[]

In [7]:
# Get header of CCD extension
header_1 = fits.getheader(image_name, 1)
header_2 = fits.getheader(image_name, 2)

In [8]:
# Try to get WCS: will raise error
try:
    wcs_1 = WCS(header_1)
except Exception as e:
    print(e)

ERROR 6 in wcsset() at line 2594 of file cextern/wcslib/C/wcs.c:
PV1_5 : Unrecognized coordinate transformation parameter.



In [9]:
# Change convention
header_0["CTYPE1"] = "RA-TPV"
header_0["CTYPE2"] = "DEC-TPV"

header_1["CTYPE1"] = "RA-TPV"
header_1["CTYPE2"] = "DEC-TPV"

In [10]:
wcs_0_tpv = WCS(header_0)
wcs_1_tpv = WCS(header_1)

In [11]:
# Show PV polynomials: from main header still empty
wcs_0_tpv.wcs.get_pv()

[]

In [12]:
# Show PV polynomials: now not empty
wcs_1_tpv.wcs.get_pv()

[(1, 0, -0.006879527402639),
 (1, 1, 1.01650122237),
 (1, 2, 0.007779393963485),
 (1, 3, 0.0),
 (1, 4, 0.001658429411009),
 (1, 5, 0.001232870290071),
 (1, 6, 0.0006421933266491),
 (1, 7, -0.02501061777044),
 (1, 8, -0.001885796980293),
 (1, 9, -0.02476149608525),
 (1, 10, -0.0005079512200027),
 (2, 0, -0.006116094003451),
 (2, 1, 1.01486402431),
 (2, 2, 0.007783886945903),
 (2, 3, 0.0),
 (2, 4, 0.001436380491668),
 (2, 5, 0.001297612914507),
 (2, 6, 0.0006046712116443),
 (2, 7, -0.02474775482412),
 (2, 8, -0.001916401228446),
 (2, 9, -0.02480256398885),
 (2, 10, -0.0004942902025637)]

In [13]:
header_1_sip = fits.getheader(image_name, 1)

# Transform header from PV to SIP
stp.pv_to_sip(header_1_sip)

/home/mkilbing/.conda/envs/shapepipe/lib/python3.9/site-packages/sip_tpv/reverse.py:77: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  apcoeffs, r, rank, s = np.linalg.lstsq(a, b)
/home/mkilbing/.conda/envs/shapepipe/lib/python3.9/site-packages/sip_tpv/reverse.py:82: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  bpcoeffs, r, rank, s = np.linalg.lstsq(a, b)


In [14]:
# Check key words
print(header_1_sip["CTYPE2"])
print(header_1["CTYPE2"])

DEC--TAN-SIP
DEC-TPV


In [15]:
wcs_1_sip = WCS(header_1_sip)

In [16]:
wcs_1_sip.wcs.get_pv()

[]

In [17]:
n_pix_x, n_pix_y = dat[1].data.shape
print(n_pix_x, n_pix_y)

4644 2112


In [18]:
# Define some coordinate pairs
x = np.array([2000, n_pix_x/2])
y = np.array([1500, n_pix_y/2])
xx, yy = np.meshgrid(x, y)
pixargs = np.vstack([xx.reshape(-1), yy.reshape(-1)]).T
print(pixargs)
print(pixargs[:,0])
print(pixargs[:,1])

[[2000. 1500.]
 [2322. 1500.]
 [2000. 1056.]
 [2322. 1056.]]
[2000. 2322. 2000. 2322.]
[1500. 1500. 1056. 1056.]


In [19]:
# Transform to world coordinates using PV
wcs_1_tpv.all_pix2world(pixargs, 1)

array([[205.70844802,  49.69450748],
       [205.72490131,  49.69446538],
       [205.70815285,  49.71724671],
       [205.72460613,  49.71720461]])

In [20]:
# Transform to world coordinates using SIP: different results
wcs_1_sip.all_pix2world(pixargs, 1)

array([[205.96952336,  49.69219154],
       [205.99483982,  49.69195117],
       [205.96933441,  49.7148398 ],
       [205.9946501 ,  49.7145905 ]])

In [21]:
# Transform sip back to pv
header_1_tpv_back = copy(header_1_sip)
stp.sip_to_pv(header_1_tpv_back)
wcs_1_tpv_back = WCS(header_1_tpv_back)

In [22]:
# Transform to world coordinates: same as SIP
wcs_1_tpv_back.all_pix2world(pixargs, 1)

array([[205.96952336,  49.69219154],
       [205.99483982,  49.69195117],
       [205.96933441,  49.7148398 ],
       [205.9946501 ,  49.7145905 ]])

In [23]:
# Compare with Erin Shepdon's utils
header_1["CTYPE1"] = "RA- -TPV"
header_1["CTYPE2"] = "DEC- -TPV"
wcs1_tpv_es = wcsutil.WCS(header_1)

In [25]:
# Transform to world coordinates: same as SIP
wcs1_tpv_es.image2sky(pixargs[:,0], pixargs[:,1])

(array([205.96952336, 205.99483982, 205.96933441, 205.9946501 ]),
 array([49.69219154, 49.69195117, 49.7148398 , 49.7145905 ]))